In [3]:
import tkinter
tkinter._test()

In [9]:
pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=b728f3fc383976412c5371d5b4cb895caf7eb68c99d0d3fb49174e6733001ea7
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\65\4f\66\bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [10]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Hello World!", ln=True, align='C')
pdf.output("test.pdf")

print("PDF generated successfully.")

PDF generated successfully.


In [5]:
pip install PyMuPDF

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB 217.9 kB/s eta 0:00:15
   ---------------------------------------- 0.0/3.2 MB 163.8 kB/s eta 0:00:20
   ---------------------------------------- 0.0/3.2 MB 163.8 kB/s eta 0:00:20
    --------------------------------------- 0.0/3.2 MB 151.3 kB/s eta 0:00:21
    --------------------------------------- 0.0/3.2 MB 151.3 kB/s eta 0:00:21
    --------------------------------------- 0.0/3.2 MB 151.3 kB/s eta 0:00:21
    --------------------------------------- 0.0/3.2 MB 151.3 kB/s eta 0:00:21
    --------------------------------------- 0.1/3.2 MB 136.5 kB/s eta 0:00:24
    --------------------------------------- 0.1/3.2 MB 136.5 kB/s eta 0:00:24
    --------------------------------------- 0.1/3.2 MB 136.5 kB/s eta 0:00:24
    ---------

In [6]:
pip install langdetect googletrans==4.0.0-rc1

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/981.5 kB 146.3 kB/s eta 0:00:07
     - ----------------------------------- 30.7/981.5 kB 146.3 kB/s eta 0:00:07
     - ----------------------------------- 30.7/981.5 kB 146.3 kB/s eta 0:00:07
     - ----------------------------------- 30.7/981.5 kB 146.3 kB/s eta 0:00:07
     - ----------------------------------- 30.7/981.5 kB 146.3 kB/s eta 0:00:07
     - ------------------------------------ 41.0/981.5 kB 78.8 kB/s eta 0:00:12
     - ------------------------------------ 41.0/981.5 kB 78.8 kB/s eta 0:00:12

In [14]:
pip install google-cloud-translate

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers torch pdfplumber fpdf

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import pdfplumber
from transformers import MarianMTModel, MarianTokenizer
from fpdf import FPDF
import os
import time

# Main Application Class
class PDFLanguageConverterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF Language Converter")
        self.root.geometry("500x400")
        self.root.configure(bg="#f5f5f5")

        # Header Label
        self.header_label = tk.Label(root, text="PDF Language Converter", font=("Helvetica", 18, "bold"), bg="#f5f5f5")
        self.header_label.pack(pady=20)

        # PDF Upload Button
        self.upload_button = tk.Button(root, text="Upload PDF", command=self.upload_pdf, font=("Helvetica", 12),
                                        bg="#4CAF50", fg="white", padx=10, pady=5)
        self.upload_button.pack(pady=10)

        # File Path Label
        self.file_label = tk.Label(root, text="No file selected", font=("Helvetica", 10), bg="#f5f5f5", fg="gray")
        self.file_label.pack(pady=5)

        # Language Selection Dropdown
        self.languages = ["Sinhala", "English", "Tamil"]
        self.language_codes = {
            "Sinhala": "si",
            "English": "en",
            "Tamil": "ta"
        }
        self.selected_language = tk.StringVar()
        self.language_dropdown = ttk.Combobox(root, textvariable=self.selected_language, values=self.languages,
                                              font=("Helvetica", 12))
        self.language_dropdown.set("Select Target Language")
        self.language_dropdown.pack(pady=10)

        # Progress Bar
        self.progress = ttk.Progressbar(root, orient="horizontal", length=400, mode="determinate")
        self.progress.pack(pady=20)

        # Translate Button
        self.translate_button = tk.Button(root, text="Translate PDF", command=self.start_translation_thread,
                                          font=("Helvetica", 12), bg="#2196F3", fg="white", padx=10, pady=5)
        self.translate_button.pack(pady=10)

        # Status Label
        self.status_label = tk.Label(root, text="", font=("Helvetica", 10), bg="#f5f5f5", fg="gray")
        self.status_label.pack(pady=10)

        # File Path
        self.file_path = None

        # Initialize translation model
        self.model_name = 'Helsinki-NLP/opus-mt-en-si'  # Example model, adjust according to target languages
        self.tokenizer = MarianTokenizer.from_pretrained(self.model_name)
        self.model = MarianMTModel.from_pretrained(self.model_name)

    def upload_pdf(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if self.file_path:
            if self.file_path.endswith('.pdf') and os.path.getsize(self.file_path) < 50 * 1024 * 1024:  # 50 MB size limit
                self.file_label.config(text=os.path.basename(self.file_path))
                messagebox.showinfo("Success", "PDF uploaded successfully!")
            else:
                messagebox.showerror("Error", "Invalid file format or size exceeds 50 MB.")
                self.file_path = None
                self.file_label.config(text="No file selected")

    def start_translation_thread(self):
        if not self.file_path:
            messagebox.showerror("Error", "Please upload a PDF file first.")
            return

        if self.selected_language.get() == "Select Target Language":
            messagebox.showerror("Error", "Please select a target language.")
            return

        # Disable UI during processing
        self.upload_button.config(state=tk.DISABLED)
        self.translate_button.config(state=tk.DISABLED)

        # Start threading for translation
        thread = threading.Thread(target=self.translate_pdf)
        thread.start()

    def translate_pdf(self):
        self.progress['value'] = 0
        self.update_progress(10)  # Progress for PDF upload
        self.status_label.config(text="Extracting text from PDF...")

        # Step 1: Extract text from PDF
        try:
            text = self.extract_text_from_pdf(self.file_path)
            if not text.strip():
                raise ValueError("Extracted text is empty")
        except Exception as e:
            self.status_label.config(text="Failed to extract text")
            messagebox.showerror("Error", f"Failed to extract text: {str(e)}")
            self.reset_ui()
            return

        self.update_progress(30)  # Progress for text extraction
        self.status_label.config(text="Translating text...")

        # Step 2: Translate text
        target_language_code = self.language_codes.get(self.selected_language.get())
        if not target_language_code:
            self.status_label.config(text="Invalid target language selected")
            self.reset_ui()
            return

        try:
            translated_text = self.translate_text(text, target_language_code)
            if translated_text is None:
                raise ValueError("Translation returned None")
            if not translated_text.strip():
                raise ValueError("Translated text is empty")
        except Exception as e:
            self.status_label.config(text="Failed to translate text")
            messagebox.showerror("Error", f"Failed to translate text: {str(e)}")
            self.reset_ui()
            return

        self.update_progress(60)  # Progress for text translation

        # Step 3: Rebuild translated PDF
        self.status_label.config(text="Rebuilding PDF...")
        output_path = self.file_path.replace(".pdf", f"_{target_language_code}.pdf")
        try:
            self.generate_translated_pdf(translated_text, output_path)
        except Exception as e:
            self.status_label.config(text="Failed to generate PDF")
            messagebox.showerror("Error", f"Failed to generate translated PDF: {str(e)}")
            self.reset_ui()
            return

        self.update_progress(100)  # Progress for PDF generation
        self.status_label.config(text=f"PDF translated and saved as {output_path}")
        messagebox.showinfo("Success", f"PDF translated and saved as {output_path}")

        # Re-enable the UI
        self.reset_ui()

    def extract_text_from_pdf(self, file_path):
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
        return text

    def translate_text(self, text, target_language_code):
        try:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True)
            translated = self.model.generate(**inputs)
            translated_text = self.tokenizer.decode(translated[0], skip_special_tokens=True)
            return translated_text
        except Exception as e:
            print(f"Translation error: {str(e)}")
            return None

    def generate_translated_pdf(self, translated_text, output_path):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Insert translated text into the new PDF
        for line in translated_text.split('\n'):
            pdf.multi_cell(0, 10, line)

        pdf.output(output_path)

    def update_progress(self, value):
        self.progress['value'] = value
        self.root.update_idletasks()

    def reset_ui(self):
        self.upload_button.config(state=tk.NORMAL)
        self.translate_button.config(state=tk.NORMAL)
        self.status_label.config(text="")
        self.progress['value'] = 0


# Main Application Loop
if __name__ == "__main__":
    root = tk.Tk()
    app = PDFLanguageConverterApp(root)
    root.mainloop()
